## SETUP 

In [1]:
#import libraries
import pandas as pd
import sqlite3

In [4]:
#df = pd.DataFrame(#data here :/)

df = pd.DataFrame({'name': ['Juan', 'Victoria', 'Mary'], \
'age': [23, 34, 43], 'city': ['Miami', 'Buenos Aries', 'Santiago']})


In [5]:
df

,name,age,city
0,Juan,23,Miami
1,Victoria,34,Buenos Aries
2,Mary,43,Santiago


In [7]:
#We will sqlite3 library and create a connection
cnn = sqlite3.connect('jupyter_sql_tutorial.db')

In [11]:
df.to_sql('people', cnn)

In [12]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [15]:
%sql sqlite:///jupyter_sql_tutorial.db

In [17]:
%%sql
SELECT *
FROM people

 * sqlite:///jupyter_sql_tutorial.db
Done.


index,name,age,city
0,Juan,23,Miami
1,Victoria,34,Buenos Aries
2,Mary,43,Santiago


In [18]:
%%sql
SELECT count(*)
FROM people

 * sqlite:///jupyter_sql_tutorial.db
Done.


count(*)
3


In [19]:
%%sql
SELECT sum(age) as 'age_sum'
FROM people

 * sqlite:///jupyter_sql_tutorial.db
Done.


age_sum
100


## Parameters walkthrough

In [20]:
#create dummy dataframe
df = pd.DataFrame({'transaction_id': ['9', '8', '7', '6', '5', '4', '3'], \
                  'user_id': ['rafa', 'roy', 'kenny', 'brendan', 'jurgen', 'roy', 'roy'],\
                  'transaction_date': ['2021-12-21', '2020-12-21', '2019-12-21',\
                                       '2018-11-21', '2017-10-21', '2019-03-02', '2010-01-01'],\
                  'amount': ['10', '15', '20', '24', '25', '31', '42']})

In [21]:
df

,transaction_id,user_id,transaction_date,amount
0,9,rafa,2021-12-21,10
1,8,roy,2020-12-21,15
2,7,kenny,2019-12-21,20
3,6,brendan,2018-11-21,24
4,5,jurgen,2017-10-21,25
5,4,roy,2019-03-02,31
6,3,roy,2010-01-01,42


In [22]:
#We will sqlite3 library and create a connection
cnn = sqlite3.connect('dummy.db')

In [29]:
df.to_sql('managers1', cnn)

In [30]:
%reload_ext sql

In [31]:
%sql sqlite:///dummy.db

In [32]:
%%sql
SELECT *
FROM managers1

 * sqlite:///dummy.db
Done.


index,transaction_id,user_id,transaction_date,amount
0,9,rafa,2021-12-21,10
1,8,roy,2020-12-21,15
2,7,kenny,2019-12-21,20
3,6,brendan,2018-11-21,24
4,5,jurgen,2017-10-21,25
5,4,roy,2019-03-02,31
6,3,roy,2010-01-01,42


In [34]:
%%sql
SELECT sum(amount) as 'spend_sum'
FROM managers1

 * sqlite:///dummy.db
Done.


spend_sum
167


In [35]:
%%sql
SELECT 
    user_id
    , count(*) as num_transactions
    , sum(amount) as total_amount
FROM
    managers1
WHERE
    user_id = 'roy'
    and transaction_date = '2019-03-02'

 * sqlite:///dummy.db
Done.


user_id,num_transactions,total_amount
roy,1,31


## For real this time

ovechal01, ovi8

1228 * 11,


In [1]:
#import libraries
import pandas as pd
import sqlite3

In [24]:
stat_handle = input('stat handle:')
stat_sheet = f'./data/{stat_handle}_stats.csv'
stat_df = pd.read_csv(stat_sheet, sep= '\t', header= 0, index_col= None)

stat handle:lehnero01


In [25]:
stat_df

,date,goals,assists,+/-,pim,shots,shot%,TOI,hits,blocks,faceoff %
0,2010-10-16,NaN,0,3,1.000,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-10-18,NaN,0,7,1.000,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-01-13,W,4,20,0.833,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-02-01,L,2,31,0.939,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-02-02,NaN,4,15,0.789,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
337,2021-12-01,L,5,28,0.848,NaN,NaN,NaN,NaN,NaN,NaN
338,2021-12-05,W,2,23,0.920,NaN,NaN,NaN,NaN,NaN,NaN
339,2021-12-08,NaN,3,7,0.700,NaN,NaN,NaN,NaN,NaN,NaN
340,2021-12-12,W,4,23,0.852,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
stat_handle = input('twitter handle:')
stat_sheet = f'./data/{stat_handle}.csv'
twitter_df = pd.read_csv(stat_sheet, sep= '\t', header= 0, index_col= None)

twitter handle:robinlehner


In [27]:
twitter_df

,date time,display name,content
0,2021-12-19 23:13:41-05:00,Robin Lehner,@HulkMorgan That was for the kid I pointed too...
1,2021-12-19 22:05:01-05:00,Robin Lehner,https://t.co/bZySoaAzNu what a road trip. Tha...
2,2021-12-18 22:03:34-05:00,Robin Lehner,@SickNotWeak It’s not ending tho guys. Corpora...
3,2021-12-18 21:06:33-05:00,Robin Lehner,@jdemers51 😂
4,2021-12-18 21:01:21-05:00,Robin Lehner,I’m ready for tomorrow. Yes yes yes. Lou house...
...,...,...,...
1456,2011-11-03 02:54:39-04:00,Robin Lehner,@jwrgodfrey soon man;) gotta wait for the righ...
1457,2011-11-03 02:45:03-04:00,Robin Lehner,@jwrgodfrey @TheBoroCop I'm sure !! Just like...
1458,2011-11-03 02:38:42-04:00,Robin Lehner,@MikeMcKenna56 thx man !! Keep up the good wor...
1459,2011-11-03 02:37:02-04:00,Robin Lehner,@TheBoroCop haha ok... Any scars tonight?;)


In [28]:
game_list = []
gm_dates = stat_df.iloc[:,0]
for gm in gm_dates:
    game_list.append(gm)
    
print(len(game_list))

342


In [29]:
import pandas as pd

data_points = []
post_game = 0
tweet_dates = twitter_df.iloc[:,0]
#tweet_times = twitter_df.iloc[:,1]
for gm in tweet_dates:
    ymd = gm[:10]
    if ymd in game_list:
        gm_time = f'{ymd} 00:00:00-05:00'
        gm_time = pd.to_datetime(gm_time)
        gm = pd.to_datetime(gm)
        if gm < gm_time:
            data_points.append(gm)
        else:
            post_game += 1


print(f'{len(data_points)} good points of data, {post_game} postgame tweets')

1 good points of data, 34 postgame tweets


In [30]:
for d in data_points:
    print(d)

2019-10-12 00:47:35-04:00


Maybe need to include puck drop times? Would require altering stat scraper, and likely slowing it down signifigantly, to the magnitute of rendering an entire new webpage in beautfiul soup per row in the dataset. For Ovechkin this would include 1228 pages! Maybe I should use selenium as an alternative. However at least for Ovechkin, no gameday tweets were scheduled for 7pm EST. BRB Looking at tweet timestamps

In [9]:
for point in data_points:
    print(point)